Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [4]:
#Update your token
STUDENT_TOKEN = 'Juan Rubiales'

There should be no need to change the code below.

In [5]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_board(self):
    return json.loads(self.state)

  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('draw')
      else:
        print('winner' if game.player == player else 'loser')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [6]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class ConnectFour(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def print_game(self):

    for row in self.get_board():
      print(row)
    print("-------------------")
    # print(self.state)

Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver"

In [7]:
def random_solver(game):
  board = game.get_board()
  while True:
      x = 0
      y = random.randint(0, len(board[0]) - 1)
      if board[x][y] == '.':
        print('x: %s, y: %s' % (x, y))
        return y
  return None

In [73]:
# Algorithm to play Connect 4 using heuristics

# Heuristic for the Connect Four game
# Maximize the number of consecutive pieces in a row for the player and minimize for the opponent
# Maximize the number of consecutive pieces in a column for the player and minimize for the opponent
# Maximize the number of consecutive pieces in a diagonal for the player and minimize for the opponent


# Optimize
def row_check(player, board, s):
    # Version of critical positions 
    version = [player, player, player, "."]
    for row in board:
        # If arr has more than 4 elements different from .
        for i in range(0, len(board) - 2):

            # Check for the different versions of critical positions
            for j in range(0, 4):
                # If the row has 4 consecutive pieces of the same player
                if row[i] == version[0] and row[i + 1] == version[1] and row[i + 2] == version[2] and row[i + 3] == version[3]:                    
                    
                    # Check if index same as s and the postion is empty
                    for k in range(4):
                        if i + k == s and row[i+k] == ".":
                            return True

                # Change the version to check for other combinations
                version = version[1:] + [version[0]]

# Check positive diagonal
def diagonal_check(player, board, s):
    # Version of critical positions
    version = [player, player, player, "."]

    # Loop through the board in a diagonal fashion
    for i in range(0, len(board) - 3):
        for j in range(0, 4):

            # Check for the different versions of critical positions
            for p in range(len(version)):           

                # If the row has 4 consecutive pieces of the same player
                if board[i][j] == version[0] and board[i + 1][j + 1] == version[1] and board[i + 2][j + 2] == version[2] and board[i + 3][j + 3] == version[3]:
                    
                    # Check if index same as s and if the position is empty
                    for k in range(0,4):
                        if j + k == s and board[i + k][j + k] == ".":
                            return True

                # Change the version to check for other combinations
                version = version[1:] + [version[0]]

    return False

# check 3 in a row in negative diagonal
def diagonal_check2(player, board, s):
    # Version of critical positions
    version = [player, player, player, "."]

    # Loop through the board in a diagonal fashion
    for i in range(0, len(board)):
        for j in range(0, 4):

            # Check for the different versions of critical positions
            for p in range(len(version)):

                # If the row has 4 consecutive pieces of the same player
                if board[i][j] == version[0] and board[i - 1][j + 1] == version[1] and board[i - 2][j + 2] == version[2] and board[i - 3][j + 3] == version[3]:
                    
                    # Check if index same as s and if the position is empty
                    for k in range(0,4):
                        if j + k == s and board[i - k][j + k] == ".":
                            return True
                
                # Change the version to check for other combinations
                version = version[1:] + [version[0]]

    return False
   


# j is the index of the last piece played
def heuristic(game, j, player, opponent):
    # Initial variables
    board = game.get_board()
    score = 0       

    # Check the column

    # Get the column of the last piece played
    column = [board[i][j] for i in range(len(board))]

    # Early stoping if the column is full
    if column[0] != '.':
        return 0

    # Calculate the score for the column for the player and the opponent
    score += heuristic_score(column, player)
    score += heuristic_score(column, opponent)

    # If the player has a move that creates a 3 in a column in chunks of 4, then it is a winning move
    for i in range(0, len(column) - 3):
        # print("column: ", column)
        if column[i] == '.' and column[i + 1] == player and column[i + 2] == player and column[i + 3] == player:
            print("wc")
            return 10000
    # Check if the opponent has a move that creates a 3 in a column in chunks of 4
    for i in range(0, len(column) - 3):
        if column[i] == '.' and column[i + 1] == opponent and column[i + 2] == opponent and column[i + 3] == opponent:
            print("lc")
            return 5000
    
    # -----------------------------

    # Check the row

    # Get the row of the last piece played
    row = board[0][j]

    # Calculate the score for the row for the player and the opponent
    score += heuristic_score(row, player)
    score += heuristic_score(row, opponent)
    # print("row: ", board[-1])

    # If the player has a move that creates a 3 in a row in chunks of 4, then it is a winning move
    if row_check(player, board, j):
        print("wr")
        return 10000

    if row_check(opponent, board, j):
        print("lr")
        return 5000

    # -----------------------------

    # Check the diagonal

    # Calculate the score for the diagonal for the player and the opponent
    score += heuristic_diagonal(board, player)
    score += heuristic_diagonal(board, opponent)


    # If the player has a move that creates a 3 in a diagonal in chunks of 4, then it is a winning move

    # Check positive diagonal for player
    if diagonal_check(player, board, j):
        print("wrdd")
        return 10000

    # Check positive diagonal for opponent
    if diagonal_check(opponent, board, j):
        print("lrdd")
        return 5000

    # Check negative diagonal for player
    if diagonal_check2(player, board, j):
        print("wrdd")
        return 10000

    # Check negative diagonal for opponent
    if diagonal_check2(opponent, board, j):
        print("lrdd")
        return 5000

    return score
    
def heuristic_score(check, player):
    # Initial variables
    score = 0
    consecutive = 0

    # Loop through the row/column
    for i in range(len(check)):

        # If the piece is the same as the player
        # Increase the consecutive counter
        if check[i] == player:
            consecutive += 1
        else:
            score += consecutive**2
            consecutive = 0
    score += consecutive**2
    return score

def heuristic_diagonal(board, player):
    # Initial variables
    score = 0
    consecutive = 0 
    
    # Check the diagonal from top left to bottom right
    for i in range(len(board)):

        # If the piece is the same as the player
        # Increase the consecutive counter
        if board[i][i] == player:
            consecutive += 1
        else:
            score += consecutive**2
            consecutive = 0
    score += consecutive**2

    # Check the diagonal from top right to bottom left
    for i in range(len(board)):

        # If the piece is the same as the player
        # Increase the consecutive counter
        if board[i][len(board) - i - 1] == player:
            consecutive += 1
        else:
            score += consecutive**2
            consecutive = 0
    score += consecutive**2    
    return score


In [86]:
# Algorithm to find the best move by using the heuristic function 
# This is an informed search algorithm that uses the heuristic function to find the best move
# Go to depth 3 and then use the heuristic function to find the best move

# Do minimax with the heuristic function
def minimax(game, depth, is_maximizing, alpha, beta):
    # Initial variables
    board = game.get_board()
    player = game.player
    opponent = game.other_player()
    best_score = 0

    # Check if the game is over
    if game is None:
        if game.winner() == player:
            return 10000
        else:
            return 0

    # Check if the depth is 0
    if depth == 0:
        return heuristic(game, 0, player, opponent)

    # Check if it is the maximizing player
    if is_maximizing:
        best_score = -100000
        for i in range(len(board[0])):
            if board[0][i] == ".":
                board[0][i] = player
                score = minimax(game, depth - 1, False, alpha, beta)
                board[0][i] = "."
                best_score = max(score, best_score)
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break
        return best_score

    # Check if it is the minimizing player
    else:
        best_score = 100000
        for i in range(len(board[0])):
            if board[0][i] == ".":
                board[0][i] = opponent
                score = minimax(game, depth - 1, True, alpha, beta)
                board[0][i] = "."
                best_score = min(score, best_score)
                beta = min(beta, best_score)
                if beta <= alpha:
                    break
        return best_score



In [87]:
def best_move(game):
    # Initial variables
    board = game.get_board()
    player = game.player
    opponent = game.other_player()
    best_score = -100000
    best_move = 0
    depth = 3
    
    # Go depth 3
    for i in range(len(board[0])):
        if board[0][i] == ".":
            board[0][i] = player
            score = minimax(game, depth - 1, False, -100000, 100000)
            board[0][i] = "."
            if score > best_score:
                best_score = score
                best_move = i
    return best_move

This code is the one that starts the game. In different games you will need to change the class (ConnectFour is the one for connect four game). In connect four, there are two game types, 'connect4' for a small board, and 'connect4big' for a big board.

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [88]:
game_loop(best_move, ConnectFour, 'connect4big', multi_player=False)

Creating new game...
Joining game with id: 3159
Playing as O
Waiting for the other player to join...
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', 'X', '.', '.']
-------------------
Making next move...
2
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
2
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
2
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
2
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
2
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
2
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
2
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
b'Valid move'
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.']
['X', '.', '.', '.', '.', '.', '.']
['O', '.', '.', '.', 'X', '.', '.']
-------------------
Making next move...
2
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
2
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0

Tests

In [ ]:
# Loop of 100 games and count the number of wins
def play_game():
    wins = 0
    for i in range(250):
        if game_loop(heuristic_solver, ConnectFour, 'connect4big', multi_player=False) == 1:
            wins += 1
    print("Wins: %s" % wins)
        
# play_game()

# 1 try on V2, 85% of wins
# 2 try on V2, 96% of wins
# 3 try on V3, 90% of wins (takes into account diagonal)


In [ ]:
game_loop(heuristic_solver, ConnectFour, 'connect4big', multi_player=True, id=309)

In [ ]:
# game_loop(random_solver, ConnectFour, 'connect4big', multi_player=False, id=None)